# Exploracion de datos

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

vuelos = 'C:\\Users\\max80\\OneDrive\\Documents\\DATATHON 2024\\Filghts TEC_Valid.csv'
compras = 'C:\\Users\\max80\\OneDrive\\Documents\\DATATHON 2024\\Sales TEC_Valid.csv'

df_vuelos = pd.read_csv(vuelos)
df_compras = pd.read_csv(compras)

n_vuelos = df_vuelos.shape[0]
n_compras = df_compras.shape[0]

vacios_vuelos = df_vuelos.isna()
vacios_vuelos = vacios_vuelos.sum()

vacios_compras = df_compras.isna()
vacios_compras = vacios_compras.sum()

display(df_vuelos)

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0
245749,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,BT,Playa,Ciudad Principal,2024-04-20 10:45:00,2024-04-20 14:00:00,178,NaN,NaN
245750,9c8970e9836d9c5ef9415bfa93c3f408,XA-VAC,AT,BT,Playa,Ciudad Principal,2023-12-22 12:50:00,2023-12-22 16:10:00,180,163.0,95.0
245751,3c15779202b13158f78e8a8afe377250,NaN,AW,BT,Playa,Ciudad Principal,2024-10-04 10:00:00,2024-10-04 13:15:00,240,NaN,NaN


In [41]:
print("La base de vuelos tiene:", n_vuelos,"lineas")
print("La base de compras tiene:", n_compras,"lineas")

La base de vuelos tiene: 245753 lineas
La base de compras tiene: 2564361 lineas


In [42]:
print(vacios_vuelos)

Flight_ID                0
Aeronave             80390
DepartureStation         1
ArrivalStation           1
Destination_Type         1
Origin_Type              1
STD                      0
STA                      0
Capacity                 0
Passengers          123525
Bookings            123525
dtype: int64


In [43]:
print(vacios_compras)

Flight_ID      0
ProductType    0
ProductName    0
Quantity       0
TotalSales     0
dtype: int64


In [44]:
total_ID = df_vuelos['Flight_ID'].nunique()
print(total_ID)

ID  = df_vuelos['Flight_ID'].value_counts()
ID = pd.DataFrame(ID)
ID = ID.sort_values(by = 'count', ascending = False)
#display(ID)

238055


In [45]:
df_vuelos['STD'] = pd.to_datetime(df_vuelos['STD'])
df_vuelos['Año'] = df_vuelos['STD'].dt.year
df_vuelos['Mes'] = df_vuelos['STD'].dt.month

df_vuelos = df_vuelos.sort_values(by = 'STD', ascending = True)

pasajeros_mes = df_vuelos.groupby(['Año', 'Mes'])['Passengers'].sum()

#plt.figure(figsize=(10, 6))
#pasajeros_mes.plot(marker='o', linestyle='-', color='burlywood', label='Total Pasajeros por mes')

#plt.xlabel('Fecha')
#plt.ylabel('Pasajeros')
#plt.title('Pasajeros 2023')
#plt.legend()
#plt.xticks(rotation=45)
#plt.grid(True)
#plt.tight_layout()
#plt.show()

# Creacion de base final

In [46]:
df_final = df_vuelos.dropna()
comprasxvuelo = df_compras[["Flight_ID","Quantity","TotalSales"]].groupby("Flight_ID").sum().reset_index()
comprasxvuelo = pd.DataFrame(comprasxvuelo)
mapeo_compras = dict(zip(comprasxvuelo["Flight_ID"], comprasxvuelo["TotalSales"]))

df_final['Time'] = df_final['STD'].dt.time
df_final['TotalSales'] = df_final["Flight_ID"].map(mapeo_compras)
df_final["Route"] = df_final["DepartureStation"] + "+" + df_final["ArrivalStation"]
df_final["Occupation fraction"] = df_final["Passengers"] / df_final["Capacity"]
#display(df_final)

C:\Users\max80\AppData\Local\Temp\ipykernel_6896\651729967.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Time'] = df_final['STD'].dt.time
C:\Users\max80\AppData\Local\Temp\ipykernel_6896\651729967.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['TotalSales'] = df_final["Flight_ID"].map(mapeo_compras)
C:\Users\max80\AppData\Local\Temp\ipykernel_6896\651729967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

# Asignacion de inputs y output

In [47]:
categorias = df_final[['Flight_ID','Route','Time']]
numericas = df_final[['Mes','TotalSales','Capacity']]

encoder = LabelEncoder()  #Codificador de categorias
categorias = categorias.apply(encoder.fit_transform) #Codificacion

X = pd.concat([pd.DataFrame(numericas), categorias], axis=1) #Inputs array
Y = df_final['Occupation fraction']  #Output

# Red neuronal de 5 capas y entrenamiento

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)  #Particion de los datos de output en entrenamiento y prueba

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Neurona de 5 capas
    tf.keras.layers.Dense(32, activation='relu'),  
    tf.keras.layers.Dense(16, activation='relu'),  
    tf.keras.layers.Dense(8, activation='relu'),   
    tf.keras.layers.Dense(4, activation='relu'),  
    tf.keras.layers.Dense(1)                      
])

model.compile(optimizer = 'adam', loss = 'mean_squared_error')   #Compilador del modelo, optimizador y funcion de error
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))  #Entrenamiento

Epoch 1/20


C:\Users\max80\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3044/3044 ━━━━━━━━━━━━━━━━━━━━ 3s 761us/step - loss: 47416.5859 - val_loss: 0.7270
Epoch 2/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 768us/step - loss: 0.6621 - val_loss: 0.3956
Epoch 3/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 725us/step - loss: 0.2828 - val_loss: 0.0446
Epoch 4/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 742us/step - loss: 0.0314 - val_loss: 0.0233
Epoch 5/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 717us/step - loss: 0.0236 - val_loss: 0.0233
Epoch 6/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 734us/step - loss: 0.0239 - val_loss: 0.0233
Epoch 7/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 736us/step - loss: 0.0236 - val_loss: 0.0233
Epoch 8/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 727us/step - loss: 0.0237 - val_loss: 0.0234
Epoch 9/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 729us/step - loss: 0.0238 - val_loss: 0.0233
Epoch 10/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 715us/step - loss: 0.0233 - val_loss: 0.0233
Epoch 11/20
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 2s 721us/step - loss: 0.0237 - val_loss: 0.0233
Epoch 12/20
304

# Prueba de errores con datos conocidos

In [52]:
prediccion_23 = model.predict(X)
prediccion_23 = prediccion_23.flatten()
df_final['Neurona'] = prediccion_23

3804/3804 ━━━━━━━━━━━━━━━━━━━━ 2s 399us/step


C:\Users\max80\AppData\Local\Temp\ipykernel_6896\1451501567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Neurona'] = prediccion_23


In [53]:
df_final['ERROR'] = (df_final['Occupation fraction'] - df_final['Neurona'])**2
error_medio = df_final['ERROR'].mean()
df_final['Passengers Prediction'] = df_final['Neurona'] * df_final['Capacity']
display(df_final)

print("El error promedio entre las predicciones es del:",error_medio*100,"%")

C:\Users\max80\AppData\Local\Temp\ipykernel_6896\3774152266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['ERROR'] = (df_final['Occupation fraction'] - df_final['Neurona'])**2
C:\Users\max80\AppData\Local\Temp\ipykernel_6896\3774152266.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Passengers Prediction'] = df_final['Neurona'] * df_final['Capacity']


,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Año,Mes,Time,TotalSales,Route,Occupation fraction,Neurona,ERROR,Passengers Prediction
6696,030f35839975b1721e2427db425ef402,XA-VAJ,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-01-02 00:20:00,2023-01-02 05:20:00,186,120.0,68.0,2023,1,00:20:00,812.0,BM+AV,0.645161,0.881233,0.055730,163.909400
178553,7b9f51daca0f560676d28aee3d5666f7,XA-VAW,BM,BA,Ciudad Principal,Ciudad Fronteriza,2023-01-02 00:30:00,2023-01-02 06:00:00,186,170.0,99.0,2023,1,00:30:00,1070.0,BM+BA,0.913978,0.881233,0.001072,163.909400
53258,d4ac6f26a94efea00002147f15d96a41,XA-VIU,BM,AO,Ciudad Principal,Ciudad Fronteriza,2023-01-02 00:35:00,2023-01-02 05:30:00,186,185.0,89.0,2023,1,00:35:00,1239.0,BM+AO,0.994624,0.881233,0.012857,163.909400
123223,555496d4217768d1cc276ffbe83235d5,XA-VXF,AT,AU,Ecoturismo,Ciudad Principal,2023-01-02 00:55:00,2023-01-02 02:45:00,240,168.0,118.0,2023,1,00:55:00,939.0,AT+AU,0.700000,0.881233,0.032846,211.496000
53248,01cb03565042d43a1deafae09eef0ca1,XA-VAN,BM,AO,Ciudad Principal,Ciudad Fronteriza,2023-01-02 01:15:00,2023-01-02 06:10:00,186,165.0,92.0,2023,1,01:15:00,241.0,BM+AO,0.887097,0.881233,0.000034,163.909400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221868,a665d50e95bf905d862563facacbf05a,XA-VXA,AO,BM,Ciudad Fronteriza,Ciudad Principal,2023-12-31 23:00:00,2024-01-01 00:05:00,240,201.0,90.0,2023,12,23:00:00,829.0,AO+BM,0.837500,0.881233,0.001913,211.496000
92561,6dda5bd834419dd5b900408b0f5fd77b,XA-VAU,BM,AT,Ciudad Principal,Ciudad Fronteriza,2023-12-31 23:05:00,2024-01-01 04:30:00,186,131.0,81.0,2023,12,23:05:00,2397.0,BM+AT,0.704301,0.881233,0.031305,163.909400
105522,c711d31ee17b65d788d2df7ebde346f1,XA-VBI,AK,AT,Ciudad Principal,Playa,2023-12-31 23:15:00,2024-01-01 00:45:00,220,217.0,126.0,2023,12,23:15:00,1446.0,AK+AT,0.986364,0.881233,0.011052,193.871334
57082,dc6b4fa09794681fbf19cdd98f5a942e,XA-VIF,BM,AO,Ciudad Principal,Ciudad Fronteriza,2023-12-31 23:45:00,2024-01-01 04:40:00,186,168.0,75.0,2023,12,23:45:00,3208.0,BM+AO,0.903226,0.881233,0.000484,163.909400


El error promedio entre las predicciones es del: 2.347835970373793 %


# Predicción de los datos para 2024